In [92]:
import pandas as pd
import geopandas as gp
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
import shapely
import math

In [93]:
import os
#had to add GDAL_DATA variable to system variables and set value to the folder of gdal in C:\Users\mishaun\AppData\Local\Continuum\anaconda3\Library\share\gdal on my work computer
'GDAL_DATA' in os.environ

False

In [94]:
#These 2 lines of code will allow for all output to be displayed within a given cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Reading in Data

In [95]:
shapezipfile = ("zip://Data/BLMWY-2020-Q1-3_WGS84.zip")
tractshp = gp.read_file(shapezipfile, encoding = "utf-8")

In [96]:
#this is shapefile downloaded from drillinginfo holding well information
prodshp = gp.read_file("zip://Data/production.ZIP")
permitshp = gp.read_file("zip://Data/permits.ZIP")

#reading in csv of leases - converting to a GeoDataFrame - initial coord system is epsg:4326
leases = pd.read_csv("Data/LeasesTable.CSV")
leasesgeo = gp.GeoDataFrame(leases, crs = {'init': 'epsg:4326'}, geometry=gp.points_from_xy(leases["Longitude (WGS84)"], leases["Latitude (WGS84)"]))

### Trimming and Cleaning Data

In [97]:
leasesgeo.columns

Index(['State/Province', 'Effective Date', 'Record Date',
       'Expiration of Primary Term', 'Term (Months)', 'Grantor',
       'Grantee Alias', 'Royalty', 'Bonus', 'Area (Acres)', 'Section',
       'Township', 'Township Direction', 'Range', 'Range Direction',
       'Vol/Page', 'Record Number', 'Instrument Type', 'Instrument Date',
       'County/Parish', 'Options/Extensions', 'DI Basin', 'Ext. Bonus',
       'Ext. Term (Months)', 'Abstract', 'Block', 'BLM', 'State Lease',
       'Grantee', 'Grantor Address', 'Grantee Address', 'Max Depth',
       'Majority Legal Assignee', 'DI Subplay', 'Min Depth',
       'Majority Assignment Effective Date', 'Latitude (WGS84)', 'DI Play',
       'Majority Legal Assignee Interest', 'Longitude (WGS84)',
       'Majority Assignment Vol/Page', 'geometry'],
      dtype='object')

In [98]:
leasesgeo.drop(columns = ['Instrument Type', 'Instrument Date','Options/Extensions', 'DI Basin', 'Ext. Bonus',
       'Ext. Term (Months)', 'Abstract', 'Block', 'BLM', 'State Lease',
       'Grantee', 'Grantor Address', 'Grantee Address', 'Max Depth',
       'Majority Legal Assignee', 'DI Subplay', 'Min Depth',
       'Majority Assignment Effective Date','Majority Legal Assignee Interest','Majority Assignment Vol/Page'], inplace = True)

In [99]:
prodshp.columns

Index(['APIUWI', 'OpAlias', 'LeaseName', 'WellNo', 'County', 'Reservoir',
       'ProdType', 'ProdStatus', 'DrillType', 'TD', 'SpudDate', 'FstPrdDate',
       'LstPrdDate', 'MoProd', 'CumGas', 'DailyGas', 'CumLiq', 'DailyLiq',
       'LatestLiq', 'LatestGas', 'CumWtr', 'CumBOE', 'DISubplay', '1moLiq',
       '1moGas', '6moLiq', 'DIBasin', '6moGas', '6moBOE', '6moWater', 'DIPlay',
       'PracIP_Liq', 'PracIP_BOE', 'PracIP_Gas', 'PrcIPCFGED', 'LatestWtr',
       'Prior12Liq', 'Prior12Gas', 'LastTestDt', 'Prior12Wtr', 'LastFlwPrs',
       'LastWHSIP', '2moGOR', 'LatestGOR', 'CumGOR', 'Lst12Yield', '2moYield',
       'LatestYld', 'PeakGas', 'PkGasMoNo', 'PeakLiq', 'PkLiqMoNo', 'PeakBOE',
       'PkBOEMoNo', 'PkMMCFGE', 'PkMMCFGMoN', 'TopPerf', 'BtmPerf', 'GasGrav',
       'OilGrav', 'CompDate', 'WellCount', 'MaxActvWel', 'GasGather',
       'LiqGather', 'LeaseNo', 'PerfLength', 'TVD', 'Field', 'State',
       'District', 'GeoProvin', 'Section', 'Country', 'Township', 'Range',
       'Lati

In [100]:
prodshp.drop(columns = ['LatestWtr','CumWtr',
       'Prior12Liq', 'Prior12Gas', 'LastTestDt', 'Prior12Wtr', 'LastFlwPrs',
       'LastWHSIP', '2moGOR', 'LatestGOR', 'CumGOR', 'Lst12Yield', '2moYield',
       'LatestYld', 'PeakGas', 'PkGasMoNo', 'PeakLiq', 'PkLiqMoNo', 'PeakBOE',
       'PkBOEMoNo', 'PkMMCFGE', 'PkMMCFGMoN', 'TopPerf', 'BtmPerf', 'GasGrav',
       'OilGrav','CompDate', 'GasGather',
       'LiqGather', 'LeaseNo'], inplace = True)

In [101]:
permitshp.columns

Index(['API10UWI', 'District', 'FiledDate', 'AprvdDate', 'ExpDate', 'State',
       'County', 'OpAlias', 'LeaseName', 'WellNo', 'Formation', 'PermDepth',
       'TVD', 'PermitType', 'WellType', 'DrillType', 'WellStatus',
       'PermStatus', 'Field', 'OpReported', 'AmendDate', 'CntctName',
       'CntctPhone', 'OperAddrs', 'OperCity', 'OperState', 'OperZip',
       'OperCity30', 'Section', 'OperCity50', 'Township', 'Range', 'Block',
       'Survey', 'TVD_UOM', 'Abstract', 'WGID', 'H2S_Area', 'Latitude',
       'Longitude', 'OFS_Reg', 'Btm_Lat', 'Btm_Lon', 'LeaseNo', 'PermDUOM',
       'PermitNo', 'DIBasin', 'DIPlay', 'DISubplay', 'OpCompany', 'OpTicker',
       'geometry'],
      dtype='object')

In [102]:
permitshp.drop(columns = ['OpReported', 'AmendDate', 'CntctName',
       'CntctPhone', 'OperAddrs', 'OperCity', 'OperState', 'OperZip',
       'OperCity30', 'Section', 'OperCity50', 'Township', 'Range', 'Block',
       'Survey', 'TVD_UOM', 'Abstract', 'WGID', 'H2S_Area','OFS_Reg', 'LeaseNo', 'PermDUOM',
       'PermitNo','OpCompany', 'OpTicker'], inplace=True)

In [103]:
leasesgeo["Record Date"] = pd.to_datetime(leasesgeo["Record Date"])
prodshp["FstPrdDate"] = pd.to_datetime(prodshp["FstPrdDate"])
permitshp["AprvdDate"] = pd.to_datetime(permitshp["AprvdDate"])

In [104]:
leasesgeo.info()
prodshp.info()
permitshp.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 53266 entries, 0 to 53265
Data columns (total 22 columns):
State/Province                53266 non-null object
Effective Date                31049 non-null object
Record Date                   53266 non-null datetime64[ns]
Expiration of Primary Term    53266 non-null object
Term (Months)                 53266 non-null int64
Grantor                       53266 non-null object
Grantee Alias                 53239 non-null object
Royalty                       33903 non-null float64
Bonus                         18347 non-null float64
Area (Acres)                  52506 non-null float64
Section                       53266 non-null float64
Township                      53266 non-null float64
Township Direction            53266 non-null object
Range                         53266 non-null float64
Range Direction               53266 non-null object
Vol/Page                      53266 non-null object
Record Number                 53266 no

### Converting geodataframes to same coord system - UTM system for creating buffers

In [105]:
prodshp.to_crs(epsg = 26913, inplace = True)
permitshp.to_crs(epsg = 26913, inplace = True)
leasesgeo.to_crs(epsg = 26913, inplace = True)
tractshp.to_crs(epsg = 26913, inplace = True)

In [106]:
prodshp.crs
permitshp.crs
leasesgeo.crs

{'init': 'epsg:26913', 'no_defs': True}

{'init': 'epsg:26913', 'no_defs': True}

{'init': 'epsg:26913', 'no_defs': True}

### Adding a column to tract shapefile data for centroids of each tract for creating buffers upon

In [107]:
tractshp["centroids"] = tractshp.centroid
#calculating acres of tract, conv to acres is m^2 to acres
tractshp['Acres'] = round(tractshp.area * 0.000247105)
tractshp.head()

,SaleParcel,lot_no,tract_id,short_code,label,geometry,centroids,Acres
0,WY-201Q-105,65012,2,BLMWY-2020-Q1-3,WY-2020-03-0374,"MULTIPOLYGON (((173527.819 4984155.461, 173534...",POINT (172183.913 4984182.822),941.0
1,WY-201Q-063,64970,16,BLMWY-2020-Q1-3,WY-2020-03-6207,"MULTIPOLYGON (((219007.722 4650891.363, 218687...",POINT (222418.619 4647579.181),1322.0
2,WY-201Q-001,64908,93,BLMWY-2020-Q1-3,WY-2020-03-6613,"POLYGON ((562927.151 4797521.250, 563330.976 4...",POINT (563130.227 4797125.862),80.0
3,WY-201Q-002,64909,102,BLMWY-2020-Q1-3,WY-2020-03-6660,"POLYGON ((514894.040 4607634.757, 515295.946 4...",POINT (515096.374 4607429.194),40.0
4,WY-201Q-003,64910,89,BLMWY-2020-Q1-3,WY-2020-03-6585,"MULTIPOLYGON (((518971.985 4775132.462, 519376...",POINT (520545.083 4773447.285),2316.0


In [165]:
#adding buffer around centroid point from tract of 3 mi (1609.34 meters = 1 mile)
miradius = 3
milesbuffer = miradius * 1609.34

tractshp["buffers"] = tractshp.centroids.apply(lambda x: x.buffer(milesbuffer,20))


# Testing Spatial Filters

In [109]:
TestT = 40
tractTest = tractshp[tractshp["tract_id"] == TestT].iloc[0]


permFiltered = permitshp.within(tractshp[tractshp["tract_id"] ==TestT]["buffers"].iloc[0])
prodFiltered = prodshp.within(tractshp[tractshp["tract_id"] ==TestT]["buffers"].iloc[0])
leasesFiltered = leasesgeo.within(tractshp[tractshp["tract_id"]==TestT]["buffers"].iloc[0])

In [110]:
permFiltered.value_counts()

False    46457
True        12
dtype: int64

In [111]:
permitstoeval = permitshp.loc[permFiltered]

In [112]:
prodtoeval = prodshp.loc[prodFiltered]

In [113]:
prodFiltered.value_counts()

False    62874
True         6
dtype: int64

In [114]:
leasesFiltered.value_counts()

False    53264
True         2
dtype: int64

In [115]:
leasestoeval = leasesgeo.loc[leasesFiltered]

In [116]:
leasestoeval.drop_duplicates("Record Number",inplace = True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Calculating distance between leases within 3 mi radius and tract's centroid

In [126]:
#function to get compass direction

def cardDir(point, tractRef):
    '''
    This function will take in 2 inputs and will calculate the cardinal direction between 2 points
    
    1.  point: a shapely Point object referring to a permit,lease, well found in spatial query  
    2.  tractRef: a shapely Point object referring to the centroid of the lease/tract for analysis
    '''
    
    #storing each coordinate in a temporary variable
    a = point.x
    b = point.y
    c = tractRef["centroids"].x
    d = tractRef["centroids"].y

    #south and west are positive direction for my axis convention
    western = c - a
    southern = d - b
    
    #calculating angle between points for determining which direction the point of data is in reference to tract centroid
    compdeg = math.atan(western/southern)
    #converts radians to degrees
    compdeg *= 57.2958
    compdeg

    if southern > 0:
        if compdeg < 20:
            carddir = "S"
        elif compdeg > 75:
            if western>0:
                carddir = "W"
            else:
                carddir = "E"
        else:
            if western>0:
                carddir = "SW"
            else:
                carddir = "SE"
    else:
        if compdeg < 2:
            carddir = "N"
        elif compdeg > 60:
            if western>0:
                carddir = "W"
            else:
                carddir = "E"
        else:
            if western>0:
                carddir = "NW"
            else:
                carddir = "NE"  
     
    return carddir           
   
    

In [123]:
#calculating distance (in miles) away lease within 3 mi radius is to tract of interest and direction of data point (permit, lease, prod)
leasestoeval["distance"] = leasestoeval["geometry"].apply(lambda x: x.distance(tractTest["centroids"])/1609.34)
leasestoeval["direction"] = leasestoeval["geometry"].apply(lambda x: cardDir(x, tractTest))

permitstoeval["distance"] = permitstoeval["geometry"].apply(lambda x: x.distance(tractTest["centroids"])/1609.34)
permitstoeval["direction"] = permitstoeval["geometry"].apply(lambda x: cardDir(x, tractTest))


prodtoeval["distance"] = prodtoeval["geometry"].apply(lambda x: x.distance(tractTest["centroids"])/1609.34)
prodtoeval["direction"] = prodtoeval["geometry"].apply(lambda x: cardDir(x, tractTest))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

### Statistical summaries: bonus by year, closest lease, permits, prod, etc

In [124]:
leasestoeval.groupby(leasestoeval["Record Date"].apply(lambda x: x.year)).describe()

Term (Months)                                                \
                    count   mean std    min    25%    50%    75%    max   
Record Date                                                               
2014                  1.0  120.0 NaN  120.0  120.0  120.0  120.0  120.0   
2019                  1.0   12.0 NaN   12.0   12.0   12.0   12.0   12.0   

            Royalty         ... Longitude (WGS84)             distance  \
              count   mean  ...               75%         max    count   
Record Date                 ...                                          
2014            1.0  0.125  ...       -105.388968 -105.388968      1.0   
2019            0.0    NaN  ...       -105.389570 -105.389570      1.0   

                                                                             
                 mean std       min       25%       50%       75%       max  
Record Date                                                                  
2014         2.483696 NaN  2.483696  2.483696  2.483696  2.483696  2.483696  
2019         2.920035 NaN  2.920035  2.920035  2.920035  2.920035  2.920035  

[2 rows x 80 columns]

In [125]:
leasestoeval["Record Date"]
leasestoeval.describe()["Bonus"]["mean"]

4777    2019-09-13
49813   2014-08-05
Name: Record Date, dtype: datetime64[ns]

3100.0

### Function to write permits around tract summary

In [ ]:
#test data for inserting into functions

permitstoeval.head()
permitstoeval["OpAlias"].value_counts()

In [275]:
grouped = permitstoeval.groupby("DrillType").describe()
grouped["PermDepth"]["count"].reset_index().iloc[0]["count"]

sfd = permitstoeval.groupby("DrillType").describe().reset_index().index
sfd

8.0

RangeIndex(start=0, stop=2, step=1)

In [148]:
permitstoeval.columns

Index(['API10UWI', 'District', 'FiledDate', 'AprvdDate', 'ExpDate', 'State',
       'County', 'OpAlias', 'LeaseName', 'WellNo', 'Formation', 'PermDepth',
       'TVD', 'PermitType', 'WellType', 'DrillType', 'WellStatus',
       'PermStatus', 'Field', 'Latitude', 'Longitude', 'Btm_Lat', 'Btm_Lon',
       'DIBasin', 'DIPlay', 'DISubplay', 'geometry', 'distance', 'direction'],
      dtype='object')

In [279]:
def writePermitSummary(geoDF):
    
    count = geoDF["API10UWI"].count()
    
    a = "In a {} mile radius there are {} active permits".format(miradius, count)
    
    byWellType = geoDF.groupby("DrillType").describe()["PermDepth"].reset_index()
    
    
    
    if len(byWellType)>1:
        
        b = ["{} permits are {} wells ".format(str(int(byWellType.iloc[i]["count"])), byWellType.iloc[i]["DrillType"]) for i in byWellType.index]
        b = "and ".join(b)
        
    else:
        b = "There are {} {} permits ".format(str(int(byWellType.iloc[0]["count"])), byWellType.iloc[0]["DrillType"])
    
    summaryText = "\n".join([a,b])
    
    return print(summaryText)
    

In [280]:
writePermitSummary(permitstoeval)

In a 3 mile radius there are 12 active permits
8 permits are H wells and 4 permits are V wells 
